In [11]:
import lsst.daf.persistence as dafPersist
import matplotlib.pyplot as plt
import numpy as np

import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
import Utils.DiaSourceTools as DSTools
from astropy.time import Time

import lsst.afw.display.ds9 as ds9

import re

from multiprocessing import Pool
from functools import partial

from astropy.table import Column
import astropy as astropy
from astropy.io import ascii
from mpi4py import MPI
import pickle

import Utils.DiaSourceTools as DSTools
import sys
import os
import shutil


In [3]:
DATADIR="/datadec/cppm/jpreyes/CFHT_Tests/tests"

butler = dafPersist.Butler(DATADIR)

days_file=open("/datadec/cppm/jpreyes/CFHT_Tests/days_D3.txt", "rb")

dates = []
for day in days_file:
    day = day.replace('\n','')
    dates.append(day)

days_file.close()


In [210]:
multi_matches = None

filters = ["i","r"]
patch="15,15"
for date in dates:
    t = Time(date)
    date_catalog = None
                #print date
    for i, f in enumerate(filters):
            if butler.datasetExists("dayDiff_class_diaSrc", dataId={"filter":f,"tract":0, "patch":patch, 'date':date}):
                    date_catalog = butler.get("dayDiff_class_diaSrc", dataId={"filter":f,"tract":0, "patch":patch, 'date':date})
                    
                    if date_catalog is not None:
                            if multi_matches is None:
                                    multi_matches= afwTable.MultiMatch(date_catalog.schema, {'mjd':'D', "filter":'D'}, radius=afwGeom.Angle(1./3600., afwGeom.degrees))
                            multi_matches.add(date_catalog, {'mjd':int(t.mjd), 'filter':int(i)})


In [211]:
results = multi_matches.finish(removeAmbiguous=False)

In [212]:
lcs = DSTools.get_light_curves_from_multimatch_results2(results)


In [213]:
stack_lcs = []
for t_lc in lcs:
   l = DSTools.build_lightcurve6(t_lc, "base_CircularApertureFlux_4_5_flux", filters)

   n_curves = len(l.group_by("bandpass").groups.indices)
   if n_curves>1 and len(l)>5:
        #l = DSTools.build_lightcurve6(t_lc, "base_CircularApertureFlux_4_5_flux", filters)

        #l = correct_flux(l, patch, filters)    
        if l is not None:
                stack_lcs.append(l)


In [224]:
nodup_lcs = remove_duplicates(stack_lcs)
stack_lcs_final = []



for lcs_arr  in nodup_lcs:
    if len(lcs_arr)>1:
            stack_lcs_final.append(join_lc(lcs_arr))
    else:
            stack_lcs_final.append(lcs_arr[0])


        Use :attr:`Row.as_void` instead. [astropy.utils.decorators]


In [233]:
print len(stack_lcs),len(stack_lcs_final)
for r in stack_lcs[0]:
    v_roid = r.as_void()
    print v_roid.tolist()
    break

696 647
(53080.0, 3.0, 'ab', nan, 3.74894 rad, inf, 0.918995 rad, 25.0, 'sdssi')


In [223]:
def remove_duplicates(lcs):

    checks = [False]*len(lcs)

    errors = 0
    amb = []
    for i, l in enumerate(lcs):
        if checks[i] == True:
            continue

        checks[i] = True
        base_det = [l]
        for j in range(i+1, len(lcs)):
            if checks[j] == True:
                continue
            l1 = {"ra":l[0]["ra"], "dec":l[0]["dec"]}
            l2 = {"ra":lcs[j][0]["ra"], "dec":lcs[j][0]["dec"]}

            if DSTools.source_distance(l1,l2)<1.0:
                base_det.append(lcs[j])
                checks[j] = True
                errors+=1

        amb.append(base_det)

    return amb

def join_lc(lc_array):
    result_lc = lc_array[0]

    for i in range(1, len(lc_array)):
        appendables = []


        for src_to_add in lc_array[i]:
            add_it = True
            for src in result_lc:

                if src["mjd"]==src_to_add["mjd"] and src["bandpass"]==src_to_add["bandpass"]:
                    add_it = False
                    break
            if add_it==True:
                appendables.append(src_to_add)

        for app in appendables:
            result_lc.add_row(app.data.tolist())
    result_lc.sort("mjd")
    return result_lc


In [134]:
print len(amb), len(def_lcs)

1026 1083


In [157]:
for i, a in enumerate(amb):
    if len(a)==2:
        print(i)
print len(a)

3
12
16
19
20
25
35
38
45
94
99
136
154
248
262
367
370
377
394
404
414
468
492
506
519
533
647
701
821
1


In [142]:
print len(a[0]), len(a[1])

142 122


In [162]:
a=amb[821]

In [169]:
count = 0

for src in a[0]:
        for src_to_add in a[1]:
            if src.get("mjd")==src_to_add.get("mjd") and src.get("filter")==src_to_add.get("filter") and src.get("base_CircularApertureFlux_3_0_flux")==src_to_add.get("base_CircularApertureFlux_3_0_flux"):
                count+=1
                
print count

5


In [166]:
len(amb[821][0]),len(amb[821][1])

(10, 7)

In [173]:
res_a = join_lc(a)


In [175]:
res_a

In [103]:
   print dot.get("base_CircularApertureFlux_3_0_flux")==dot2.get("base_CircularApertureFlux_3_0_flux")

Schema(
    (Field['L'](name="id", doc="unique ID"), Key<L>(offset=0, nElements=1)),
    (Field['Angle'](name="coord_ra", doc="position in ra/dec"), Key<Angle>(offset=8, nElements=1)),
    (Field['Angle'](name="coord_dec", doc="position in ra/dec"), Key<Angle>(offset=16, nElements=1)),
    (Field['L'](name="parent", doc="unique ID of parent source"), Key<L>(offset=24, nElements=1)),
    (Field['Flag'](name="ip_diffim_NaiveDipoleCentroid_flag", doc="general failure flag, set if anything went wrong"), Key['Flag'](offset=32, bit=0)),
    (Field['Flag'](name="ip_diffim_NaiveDipoleCentroid_pos_flag", doc="failure flag for positive, set if anything went wrong"), Key['Flag'](offset=32, bit=1)),
    (Field['Flag'](name="ip_diffim_NaiveDipoleCentroid_neg_flag", doc="failure flag for negative, set if anything went wrong"), Key['Flag'](offset=32, bit=2)),
    (Field['D'](name="ip_diffim_NaiveDipoleCentroid_x", doc="unweighted first moment centroid: overall centroid", units="pixel"), Key<D>(offset

In [62]:
len(amb[5][0]), len(amb[5][1])

(12, 11)

In [49]:
sr = amb[0][0][0]
sr2 = amb[0][1][0]

In [51]:
sr["filter"],sr2["filter"]

(0.0, 0.0)

In [67]:
for sr_pair in amb[5][0]:
    print sr_pair["coord_ra"],sr_pair["coord_dec"]

3.7488 rad 0.919147 rad
3.74881 rad 0.919148 rad
3.7488 rad 0.919148 rad
3.7488 rad 0.919147 rad
3.7488 rad 0.919149 rad
3.7488 rad 0.919147 rad
3.7488 rad 0.919147 rad
3.7488 rad 0.919147 rad
3.7488 rad 0.919147 rad
3.7488 rad 0.919146 rad
3.7488 rad 0.919148 rad
3.7488 rad 0.919147 rad


In [68]:
for sr_pair in amb[5][1]:
    print sr_pair["coord_ra"],sr_pair["coord_dec"]

3.7488 rad 0.919148 rad
3.7488 rad 0.919147 rad
3.7488 rad 0.919149 rad
3.7488 rad 0.919147 rad
3.7488 rad 0.919147 rad
3.74881 rad 0.919148 rad
3.7488 rad 0.919147 rad
3.7488 rad 0.919147 rad
3.7488 rad 0.919146 rad
3.7488 rad 0.919148 rad
3.7488 rad 0.919147 rad


In [241]:
import glob
lienObj = glob.glob("/datadec/cppm/jpreyes/CFHT_Tests/light_curves_noDup/15,15*.dat")
lcs = []
for oj in lienObj:
    table = ascii.read(oj)
    lcs.append(table)
    
len(lcs)

1095

In [238]:
s_candidates= []
for lc in lcs:
    t_by_filter = lc.group_by("bandpass")

    indices = t_by_filter.groups.indices
    if len(indices)>=2:
        s_candidates.append(lc)

In [239]:
nodup_lcs = remove_duplicates(s_candidates)
stack_lcs_final = []



for lcs_arr  in nodup_lcs:
    if len(lcs_arr)>1:
            stack_lcs_final.append(join_lc(lcs_arr))
    else:
            stack_lcs_final.append(lcs_arr[0])


        Use :attr:`Row.as_void` instead. [astropy.utils.decorators]


In [240]:
len(stack_lcs_final)

1094